# 如何创建动态（自构建）链
:::info 前提条件
本指南假定您熟悉以下内容：- [LangChain 表达式语言 (LCEL)](/docs/concepts/lcel)- [如何将任意函数转换为可运行函数](/docs/how_to/functions)
:::
有时我们希望根据链的输入在运行时动态构建链的某些部分（[路由](/docs/how_to/routing/) 就是最常见的例子）。我们可以利用 RunnableLambda 的一个非常有用的特性来创建这样的动态链：如果 RunnableLambda 返回一个 Runnable，那么该 Runnable 会被自动调用。让我们来看一个示例。
import ChatModelTabs from "@theme/ChatModelTabs";
<ChatModelTabs
customVarName="llm"/>

In [4]:
# | echo: false

from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(model="claude-3-sonnet-20240229")

In [10]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import Runnable, RunnablePassthrough, chain

contextualize_instructions = """Convert the latest user question into a standalone question given the chat history. Don't answer the question, return the question and nothing else (no descriptive text)."""
contextualize_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_instructions),
        ("placeholder", "{chat_history}"),
        ("human", "{question}"),
    ]
)
contextualize_question = contextualize_prompt | llm | StrOutputParser()

qa_instructions = (
    """Answer the user question given the following context:\n\n{context}."""
)
qa_prompt = ChatPromptTemplate.from_messages(
    [("system", qa_instructions), ("human", "{question}")]
)


@chain
def contextualize_if_needed(input_: dict) -> Runnable:
    if input_.get("chat_history"):
        # NOTE: This is returning another Runnable, not an actual output.
        return contextualize_question
    else:
        return RunnablePassthrough() | itemgetter("question")


@chain
def fake_retriever(input_: dict) -> str:
    return "egypt's population in 2024 is about 111 million"


full_chain = (
    RunnablePassthrough.assign(question=contextualize_if_needed).assign(
        context=fake_retriever
    )
    | qa_prompt
    | llm
    | StrOutputParser()
)

full_chain.invoke(
    {
        "question": "what about egypt",
        "chat_history": [
            ("human", "what's the population of indonesia"),
            ("ai", "about 276 million"),
        ],
    }
)

"According to the context provided, Egypt's population in 2024 is estimated to be about 111 million."

关键在于 `contextualize_if_needed` 返回的是另一个可运行对象（Runnable），而非实际输出结果。当整个链式流程执行时，这个被返回的可运行对象才会真正运行。
观察追踪记录可以发现，由于我们传入了chat_history参数，我们在完整链中执行了contextualize_question链：https://smith.langchain.com/public/9e0ae34c-4082-4f3f-beed-34a2a2f4c991/r

请注意，返回的 Runnable 的流式处理、批处理等功能均保持不变

In [11]:
for chunk in contextualize_if_needed.stream(
    {
        "question": "what about egypt",
        "chat_history": [
            ("human", "what's the population of indonesia"),
            ("ai", "about 276 million"),
        ],
    }
):
    print(chunk)

What
 is
 the
 population
 of
 Egypt
?
